## The Data - Lisbon Neighborhoods

For this assignement the data was collected from a website:

https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=

In [1]:
# libraries to import
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [19]:
PageToScrape = requests.get('https://services.arcgis.com/1dSrzEWVQn5kHHyK/arcgis/rest/services/Limites_Cartografia/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPoint&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=true&outFields=*&returnGeometry=false&returnCentroid=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=html&token=').text
PageToScrape = BeautifulSoup(PageToScrape, 'xml')
PageToScrape = PageToScrape.find_all('table',{'class':'ftrTable'})

In [20]:
table = []
for row in PageToScrape:
    table.append([t.text.strip() for t in row.find_all('td')])
#df = pd.DataFrame(table, columns=['OBJECTID', 'NOME'])
df = pd.DataFrame(table)
df = df.drop([0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], axis=1)
df.head()

,5,21
0,Santo António,Point:\nX: -1018440.6546671605\nY: 4681834.853...
1,Parque das Nações,Point:\nX: -1012503.9786498934\nY: 4689466.068...
2,Marvila,Point:\nX: -1014226.7638454303\nY: 4685629.513...
3,Ajuda,Point:\nX: -1023989.21146323\nY: 4680527.21680...
4,Areeiro,Point:\nX: -1016739.3330696875\nY: 4684693.942...


Form this page I was able to get the parishes of Lisbo and there centroid, not scrapped. The next step was to convert to X and Y associated with each parish.

In [21]:
df[21] = df[21].str.replace("Point:\n", "")
# new data frame with split value columns 
new = df[21].str.split("\n", n = 1, expand = True) 
# making separate first name column from new data frame 
df["X"]= new[0] 
# making separate last name column from new data frame 
df["Y"]= new[1]
# Dropping old Name columns 
df.drop(columns =[21], inplace = True) 
df["X"] = df["X"].str.replace("X: ", "")
df["Y"] = df["Y"].str.replace("Y: ", "")
df.rename(columns={5: 'Freguesia'}, inplace=True)
df.head()

,Freguesia,X,Y
0,Santo António,-1018440.6546671605,4681834.85372911
1,Parque das Nações,-1012503.9786498934,4689466.0681953141
2,Marvila,-1014226.7638454303,4685629.5130308038
3,Ajuda,-1023989.21146323,4680527.2168011209
4,Areeiro,-1016739.3330696875,4684693.9425387029


I got a X and Y but I needed to convert to the spacial reference for GPS.

In [22]:
!pip install pyproj
from pyproj import Proj, transform

In [23]:
def function(row):
    inProj = Proj(init='epsg:3857')
    outProj = Proj(init='epsg:4326')
    z2 = transform(inProj,outProj,row.X, row.Y)
    return z2
df['GPS'] = df.apply(function, axis=1)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/opt/conda/envs/Python36/lib/python3.6/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.

In [24]:
df1 = df
df1[["GPS"]] = df1[["GPS"]].astype("str")
df1['GPS'] = df1['GPS'].str.replace("(", "")
df1['GPS'] = df1['GPS'].str.replace(")", "")
#df1.head()
#df1.dtypes
new = df1['GPS'].str.split(",", n = 1, expand = True) 
df1.head()
# making separate first name column from new data frame 
df1["Longitude"]= new[0] 
df1.head()
#df1.head()
# making separate last name column from new data frame 
df1["Latitude"]= new[1]
# Dropping old Name columns 
df1.drop(columns =['GPS'], inplace = True) 
df1 = df1.drop(['X', 'Y'], axis=1)
df1.head()

,Freguesia,Longitude,Latitude
0,Santo António,-9.148808060562017,38.72134487346995
1,Parque das Nações,-9.095477992530249,38.774809235541774
2,Marvila,-9.110954035254304,38.74793529763537
3,Ajuda,-9.19865159430916,38.712179538844204
4,Areeiro,-9.13352482861989,38.741380367314136
